In [ ]:
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
air_quality = pd.read_csv("data/air_quality_no2_long.csv")

air_quality = air_quality.rename(columns={"date.utc": "datetime"})

air_quality.head()

In [ ]:
air_quality.city.unique()

[Fonte](https://pandas.pydata.org/docs/getting_started/intro_tutorials/09_timeseries.html)

## Usando propriedades de data e hora do pandas

Eu quero trabalhar com as datas na coluna `datetime` como objetos de data e hora em vez de texto simples

In [ ]:
air_quality["datetime"] = pd.to_datetime(air_quality["datetime"])

air_quality["datetime"]

Inicialmente, os valores em `datetime` são cadeias de caracteres e não fornecem nenhuma operação de data/hora (por exemplo, extrair o ano, dia da semana,…). Para aplicar a função `to_datetime`, o pandas interpreta as strings e as converte em objetos `datetime` (isto é, `.datetime64[ns, UTC]`). No Pandas, chamamos esses objetos de data e hora semelhantes aos da biblioteca padrão `datetime.datetime` como `pandas.Timestamp`.

> Como muitos conjuntos de dados contêm informações de data e hora em uma das colunas, a função de entrada do pandas [`pandas.read_csv()`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html#pandas.read_csv) e [`pandas.read_json()`](https://pandas.pydata.org/docs/reference/api/pandas.read_json.html#pandas.read_json) pode fazer a transformação em datas ao ler os dados usando o parâmetro `parse_dates` com uma lista das colunas a serem lidas como Timestamp:

```
pd.read_csv("data/air_quality_no2_long.csv", parse_dates=["datetime"])

```

Por que esses objetos [`pandas.Timestamp`](https://pandas.pydata.org/docs/reference/api/pandas.Timestamp.html#pandas.Timestamp) são úteis? Vamos ilustrar o valor agregado com alguns casos de exemplo.

Qual é a data inicial e final do conjunto de dados da série temporal com o qual estamos trabalhando?

In [ ]:
air_quality["datetime"].min(), air_quality["datetime"].max()

Usar [`pandas.Timestamp`](https://pandas.pydata.org/docs/reference/api/pandas.Timestamp.html#pandas.Timestamp) para `datetimes` nos permite calcular com informações de data e torná-las comparáveis. Portanto, podemos usar isso para obter o comprimento de nossa série temporal:

In [ ]:
air_quality["datetime"].max() - air_quality["datetime"].min()

O resultado é um objeto [`pandas.Timedelta`](https://pandas.pydata.org/docs/reference/api/pandas.Timedelta.html#pandas.Timedelta), semelhante ao `datetime.timedelta` da biblioteca padrão Python e definindo uma duração de tempo.

> Os vários conceitos de tempo suportados pelos pandas são explicados na seção do guia do usuário sobre [conceitos relacionados ao tempo](https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-overview).

Quero adicionar uma nova coluna ao DataFramecontendo apenas o mês da medição

In [ ]:
air_quality["month"] = air_quality["datetime"].dt.month

air_quality.head()

Ao usar objetos `Timestamp` para datas, muitas propriedades relacionadas ao tempo são fornecidas pelos pandas. Por exemplo, o `month`, mas também `year`, `quarter`,… Todas essas propriedades são acessíveis pelo `dt` acessador.

> Uma visão geral das propriedades de data existentes é fornecida na [tabela de visão geral dos componentes de data e hora](https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-components). Mais detalhes sobre o acessador `dt` para retornar propriedades semelhantes a data e hora são explicados em uma seção dedicada no [acessador dt](https://pandas.pydata.org/docs/user_guide/basics.html#basics-dt-accessors) .

Qual é a média de concentração ***NO<sub>2<sub>*** para cada dia da semana para cada um dos locais de medição?

> Lembra-se do padrão _split-apply-combine_ fornecido pelo `groupby` no tutorial sobre [cálculo de estatísticas](https://pandas.pydata.org/docs/getting_started/intro_tutorials/06_calculate_statistics.html#min-tut-06-stats)? Aqui, queremos calcular uma determinada estatística (por exemplo, média NO<sub>2</sub>) para cada dia da semana e para cada local de medição. Para agrupar em dias de semana, usamos a propriedade datetime `weekday` (com Monday=0 e Sunday=6) do pandas `Timestamp`, que também é acessível pelo acessador `dt`. O agrupamento em locais e dias da semana pode ser feito para dividir o cálculo da média em cada uma dessas combinações.

> Como estamos trabalhando com uma série temporal muito curta nesses exemplos, a análise não fornece um resultado representativo de longo prazo!

In [ ]:
air_quality.groupby([air_quality["datetime"].dt.weekday, "location"])["value"].mean()

Traçar o típico NO<sub>2</sub> padrão durante o dia de nossa série temporal de todas as estações juntas. Em outras palavras, qual é o valor médio para cada hora do dia?

In [ ]:
fig, axs = plt.subplots(figsize=(12, 4))

air_quality.groupby(air_quality["datetime"].dt.hour)["value"].mean().plot(
    kind='bar', rot=0, ax=axs
)

# custom x label using Matplotlib
plt.xlabel("Hora do dia")  
plt.ylabel("$NO_2 (µg/m^3)$")

Semelhante ao caso anterior, queremos calcular uma determinada estatística (por exemplo, média NO<sub>2</sub>) para cada hora do dia e podemos usar a abordagem dividir-aplicar-combinar novamente. Para este caso, usamos a propriedade datetime `hour` do pandas `Timestamp`, que também pode ser acessada pelo acessador `dt`.

## Data e hora como índice 

No [tutorial sobre remodelagem](https://pandas.pydata.org/docs/getting_started/intro_tutorials/07_reshape_table_layout.html#min-tut-07-reshape), [`pivot()`](https://pandas.pydata.org/docs/reference/api/pandas.pivot.html#pandas.pivot) foi introduzido para remodelar a tabela de dados com cada um dos locais de medição como uma coluna separada:

In [ ]:
no_2 = air_quality.pivot(index="datetime", columns="location", values="value")
no_2.head()

> Ao dinamizar os dados, as informações de data e hora se tornaram o índice da tabela. Em geral, definir uma coluna como um índice pode ser obtido pela função `set_index`.

Trabalhar com um índice de data e hora (ou seja, `DatetimeIndex` ) fornece funcionalidades poderosas. Por exemplo, não precisamos do acessador `dt` para obter as propriedades da série temporal, mas temos essas propriedades disponíveis diretamente no índice:

In [ ]:
no_2.index.year, no_2.index.weekday

Algumas outras vantagens são o subconjunto conveniente do período de tempo ou a escala de tempo adaptada nas parcelas. Vamos aplicar isso em nossos dados.

Crie uma plot dos valores do NO<sub>2</sub> nas diferentes estações de 20 de maio até o final de 21 de maio

In [ ]:
no_2["2019-05-20":"2019-05-21"].plot();

Ao fornecer uma string que analisa um datetime, um subconjunto específico dos dados pode ser selecionado em um arquivo `DatetimeIndex`.

> Mais informações sobre o `DatetimeIndex` e o fatiamento usando strings são fornecidas na seção sobre [indexação de séries temporais](https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-datetimeindex).

## Reamostrar uma série temporal para outra frequência

Agregue os valores atuais da série temporal horária ao valor máximo mensal em cada uma das estações.

> Um método muito poderoso em dados de séries temporais com um índice de data e hora é a capacidade de [`resample()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.resample.html#pandas.Series.resample) séries temporais para outra frequência (por exemplo, converter dados secundários em dados de 5 minutos).

O método [`resample()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.resample.html#pandas.Series.resample) é semelhante a uma operação groupby:

* ele fornece um agrupamento baseado em tempo, usando uma string (por exemplo `M`, `5H`, ,…) que define a frequência alvo

* requer uma função de agregação como `mean`, `max`,…

In [ ]:
monthly_max = no_2.resample("M").max()

monthly_max

> Uma visão geral dos aliases usados ​​para definir freqüências de séries temporais é fornecida na tabela de [visão geral de aliases de deslocamento](https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases).

Quando definida, a frequência da série temporal é fornecida pelo atributo `freq`:

In [ ]:
monthly_max.index.freq

Faça um gráfico da média diária NO<sub>2</sub> valor em cada uma das estações.

In [ ]:
no_2.resample("D").mean().plot(style="-o", figsize=(10, 5));

Mais detalhes sobre o poder da série temporal `resampling` são fornecidos na seção do guia do usuário sobre [reamostragem](https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-resampling).

## LEMBRAR

* Strings de data válidas podem ser convertidas em objetos datetime usando função `to_datetime` ou como parte de funções de leitura.

* Os objetos Datetime em pandas suportam cálculos, operações lógicas e propriedades convenientes relacionadas à data usando o acessador `dt`.

* A `DatetimeIndex` contém essas propriedades relacionadas à data e oferece suporte a fatias convenientes.

* `Resample` é um método poderoso para alterar a frequência de uma série temporal.


> Uma visão geral completa sobre séries temporais é fornecida nas páginas sobre [séries temporais e funcionalidade de data](https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries).